In [6]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [7]:
class DNN(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super().__init__()
        self.l1=nn.Linear(input_dim,hidden_dim)
        self.a1=nn.Tanh()
        self.l2=nn.Linear(hidden_dim,hidden_dim)
        self.a2=nn.Tanh()
        self.l3=nn.Linear(hidden_dim,hidden_dim)
        self.a3=nn.Tanh()
        self.l4=nn.Linear(hidden_dim,output_dim)
        
        self.layers=[self.l1,self.a1,self.l2,self.a2,self.l3,self.a3,self.l4]
    
    def forward(self,x):
        for layer in self.layers:
            x=layer(x)
        return x        

In [8]:
root=os.path.join('~','.torch','mnist')
transform=transforms.Compose([transforms.ToTensor(),lambda x: x.view(-1)])

mnist_train=datasets.MNIST(root=root,download=True,train=True,transform=transform)
mnist_test=datasets.MNIST(root=root,download=True,train=False,transform=transform)

train_dataloader=DataLoader(mnist_train,batch_size=100,shuffle=True)
test_dataloader=DataLoader(mnist_test,batch_size=100,shuffle=False)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=DNN(784,200,10).to(device)

In [11]:
criterion=nn.CrossEntropyLoss()
optimizer=optimizers.SGD(model.parameters(),lr=0.01)

def compute_loss(t,y):
    return criterion(y,t)

def train_step(x,t):
    model.train()
    preds=model(x)
    loss=compute_loss(t,preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss,preds

epochs=30
for epoch in range(epochs):
    train_loss=0.
    train_acc=0.
    for (x,t) in train_dataloader:
        x,t=x.to(device),t.to(device)
        loss,preds=train_step(x,t)
        train_loss+=loss.item()
        train_acc+=accuracy_score(t.tolist(),preds.argmax(dim=-1).tolist())
    train_loss/=len(train_dataloader)
    train_acc/=len(train_dataloader)
        
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch+1,train_loss,train_acc))

epoch: 1, loss: 1.801, acc: 0.559
epoch: 2, loss: 0.788, acc: 0.786
epoch: 3, loss: 0.522, acc: 0.857
epoch: 4, loss: 0.419, acc: 0.884
epoch: 5, loss: 0.369, acc: 0.897
epoch: 6, loss: 0.340, acc: 0.904
epoch: 7, loss: 0.320, acc: 0.909
epoch: 8, loss: 0.304, acc: 0.913
epoch: 9, loss: 0.292, acc: 0.916
epoch: 10, loss: 0.281, acc: 0.919
epoch: 11, loss: 0.272, acc: 0.922
epoch: 12, loss: 0.263, acc: 0.924
epoch: 13, loss: 0.255, acc: 0.926
epoch: 14, loss: 0.248, acc: 0.928
epoch: 15, loss: 0.241, acc: 0.931
epoch: 16, loss: 0.233, acc: 0.932
epoch: 17, loss: 0.227, acc: 0.934
epoch: 18, loss: 0.220, acc: 0.936
epoch: 19, loss: 0.213, acc: 0.938
epoch: 20, loss: 0.206, acc: 0.941
epoch: 21, loss: 0.200, acc: 0.942
epoch: 22, loss: 0.194, acc: 0.944
epoch: 23, loss: 0.188, acc: 0.946
epoch: 24, loss: 0.182, acc: 0.948
epoch: 25, loss: 0.176, acc: 0.949
epoch: 26, loss: 0.171, acc: 0.951
epoch: 27, loss: 0.166, acc: 0.952
epoch: 28, loss: 0.161, acc: 0.954
epoch: 29, loss: 0.157, acc: 

In [12]:
def test_step(x,t):
    model.eval()
    preds=model(x)
    loss=criterion(preds,t)
    return loss,preds

test_loss=0.
test_acc=0.

for (x,t) in test_dataloader:
    x,t=x.to(device),t.to(device)
    loss,preds=test_step(x,t)
    test_loss+=loss.item()
    test_acc+=accuracy_score(t.tolist(),preds.argmax(dim=-1).tolist())
test_loss/=len(test_dataloader)
test_acc/=len(test_dataloader)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(test_loss,test_acc))

test_loss: 0.159, test_acc: 0.954
